# Extraire des données de HAL

Extraire les données

In [ ]:
import requests
import json

rows = 1000
url = "https://api.archives-ouvertes.fr/search/"

params = {
    "q": "collCode_s:CREST",
    "fl": "*",
    "rows": rows,
    "wt": "json"
}

all_docs = []
start = 0

while True:
    params["start"] = start
    params["rows"] = rows

    r = requests.get(url, params=params)
    r.raise_for_status()

    resp = r.json()
    docs = resp["response"]["docs"]

    if not docs:
        break

    all_docs.extend(docs)
    start += rows
    print(f"Fetched {len(docs)} documents, total so far: {len(all_docs)}")

with open("hal_crest_data.json", "w", encoding="utf-8") as f:
    json.dump(all_docs, f, ensure_ascii=False, indent=4)

print(f"Total documents fetched: {len(all_docs)}")

Fetched 1000 documents, total so far: 1000
Fetched 1000 documents, total so far: 2000
Fetched 891 documents, total so far: 2891
Total documents fetched: 2891


notions non introduites : boucle, fonction

Regarder un document

In [1]:
#all_docs[0]

In [23]:
with open("un element.json", "w") as f:
    json.dump(all_docs[19], f, ensure_ascii=False, indent=4)

## Extraire l'information utile

Faire une fonction qui : 
- prend un élément
- renvoie le titre, l'abstract, la date de publication, le type de document et la liste des auteurs

In [3]:
import json
with open("hal_crest_data.json", "r") as f:
    data = json.load(f)
len(data)

2891

In [5]:
#data[0]

In [6]:
def extract_info(doc):
    """
    Extraire les données
    """
    return {
        "docid": doc.get("docid"),
        "title": doc.get("title_s", ""),
        "abstract": doc.get("abstract_s", ""),
        "abstract_eng": doc.get("en_abstract_s", ""),
        "publicationDate_tdate": doc.get("publicationDate_tdate", ""),
        "document_type": doc.get("primaryDomain_s", ""),
        "authors": doc.get("authFullName_s", []),
    }

- title_s
- abstract_s ; en_abstract_s
- publicationDate_tdate
- authFullName_s
- primaryDomain_s
- domain_s
- journalTitle_s

In [24]:
import pandas as pd
corpus = [extract_info(doc) for doc in data]
df = pd.DataFrame(corpus)
df.set_index("docid", inplace=True)
df.head()

,title,abstract,abstract_eng,publicationDate_tdate,document_type,authors
docid,,,,,,
3707033,[Unemployment and Online Labor - Evidence from...,,,2022-06-13T00:00:00Z,shs.eco,"[Ulrich Laitenberger, Steffen Viete, Olga Sliv..."
3026577,[The effect of price magnitude on analysts' fo...,[Recent research in finance shows that the mag...,[Recent research in finance shows that the mag...,2021-01-01T00:00:00Z,shs.eco,"[Tristan Roger, Wael Bousselmi, Patrick Roger,..."
4358212,[Model-based Clustering with Missing Not At Ra...,"[Model-based unsupervised learning, as any lea...","[Model-based unsupervised learning, as any lea...",2024-06-18T00:00:00Z,stat.ml,"[Aude Sportisse, Matthieu Marbac, Fabien Lapor..."
5444020,[Stationarity and ergodic properties for some ...,,,2023-12-01T00:00:00Z,phys.phys.phys-data-an,"[Paul Doukhan, Lionel Truquet, Michael H. Neum..."
4996040,[Inferring the parameters of Taylor’s power la...,[Taylor’s power law of fluctuation scaling has...,[Taylor’s power law of fluctuation scaling has...,2025-11-26T00:00:00Z,stat,"[Lionel Truquet, Joel E. Cohen, Paul Doukhan]"


Mettre en forme 
- ne garder que les éléments contenant un abstract
- mettre en forme les dates

In [29]:
df_ss = df[df["abstract"].notna() & (df["abstract"] != "")]
df_ss["date"] = pd.to_datetime(df_ss["publicationDate_tdate"], errors='coerce').dt.date
df_ss.to_excel("data_hal_crest.xlsx")
df_ss.shape

/var/folders/f9/d2d_05ws5gncmml0fx0c00kw0000gp/T/ipykernel_16782/1519487582.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ss["date"] = pd.to_datetime(df_ss["publicationDate_tdate"], errors='coerce').dt.date


(2308, 7)

## Quelques notions importantes sur les chaînes de caractères en Python